# 语言模型
:label:`sec_language-model`

在 :numref:`sec_text-sequence` 中，我们看到了如何将文本序列映射为标记，这些标记可以被视为一系列离散的观察值，例如单词或字符。假设长度为 $T$ 的文本序列中的标记依次为 $x_1, x_2, \ldots, x_T$。
*语言模型* 的目标是估计整个序列的联合概率：

$$P(x_1, x_2, \ldots, x_T)，$$

其中可以应用 :numref:`sec_sequence` 中的统计工具。

语言模型非常有用。例如，一个理想的语言模型应该能够通过一次生成一个标记 $x_t \sim P(x_t \mid x_{t-1}, \ldots, x_1)$ 来自动生成自然文本。
与使用打字机的猴子不同，这种模型生成的所有文本都应被视为自然语言，例如英语文本。此外，仅通过以前的对话片段来调整文本就足以生成有意义的对话。
显然，我们距离设计这样的系统还很遥远，因为它需要*理解*文本而不仅仅是生成语法上合理的文本。

尽管如此，即使在其有限的形式下，语言模型也非常有用。
例如，“to recognize speech” 和 “to wreck a nice beach” 听起来非常相似。
这会导致语音识别中的歧义，
可以通过拒绝第二种翻译的语言模型轻松解决。
同样，在文档摘要算法中，
知道“狗咬人”比“人咬狗”更常见是有价值的，或者“I want to eat grandma”是一个相当令人不安的陈述，而“I want to eat, grandma”则温和得多。

In [1]:
import torch
from d2l import torch as d2l

## 学习语言模型

显然的问题是我们应该如何对文档，甚至一系列标记进行建模。
假设我们在单词级别对文本数据进行分词。
让我们从应用基本的概率规则开始：

$$P(x_1, x_2, \ldots, x_T) = \prod_{t=1}^T P(x_t  \mid  x_1, \ldots, x_{t-1})$$

例如，
包含四个单词的文本序列的概率将给出为：

$$\begin{aligned}&P(\textrm{deep}, \textrm{learning}, \textrm{is}, \textrm{fun}) \\
=&P(\textrm{deep}) P(\textrm{learning}  \mid  \textrm{deep}) P(\textrm{is}  \mid  \textrm{deep}, \textrm{learning}) P(\textrm{fun}  \mid  \textrm{deep}, \textrm{learning}, \textrm{is})\end{aligned}$$

### 马尔可夫模型和$n$-gram
:label:`subsec_markov-models-and-n-grams`

在:numref:`sec_sequence`中的那些序列模型分析中，让我们将马尔可夫模型应用于语言建模。
如果一个序列上的分布满足一阶马尔可夫性质，即$P(x_{t+1} \mid x_t, \ldots, x_1) = P(x_{t+1} \mid x_t)$。更高阶对应于更长的依赖关系。这导致我们可以应用许多近似方法来建模一个序列：

$$
\begin{aligned}
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2) P(x_3) P(x_4),\\
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2  \mid  x_1) P(x_3  \mid  x_2) P(x_4  \mid  x_3),\\
P(x_1, x_2, x_3, x_4) &=  P(x_1) P(x_2  \mid  x_1) P(x_3  \mid  x_1, x_2) P(x_4  \mid  x_2, x_3).
\end{aligned}
$$

涉及一个、两个和三个变量的概率公式通常分别称为*unigram*、*bigram*和*trigram*模型。
为了计算语言模型，我们需要计算
词语的概率以及给定前几个词语时某个词语的条件概率。
请注意，
这样的概率是
语言模型参数。

### 词频

这里，
我们假设训练数据集是一个大型文本语料库，比如所有
维基百科条目，[Project Gutenberg](https://en.wikipedia.org/wiki/Project_Gutenberg)，
以及网上发布的所有文本。
词语的概率可以通过训练数据集中特定词语的相对词频来计算。
例如，估计$\hat{P}(\textrm{deep})$可以计算为以单词"deep"开头的任何句子的概率。稍微不那么准确的方法是统计单词"deep"出现的所有次数并除以语料库中的总词数。
这种方法对于常见词来说效果相当好。接下来，我们可以尝试估计

$$\hat{P}(\textrm{learning} \mid \textrm{deep}) = \frac{n(\textrm{deep, learning})}{n(\textrm{deep})}$$

其中$n(x)$和$n(x, x')$分别是单个词和连续词对的出现次数。
不幸的是，
估计词对的概率要困难得多，因为
"deep learning"的出现频率要低得多。
特别是对于一些不常见的词组合，可能很难找到足够的出现次数来获得准确的估计。
正如:numref:`subsec_natural-lang-stat`中的实证结果所建议的那样，对于三词组合及以上的估计情况会变得更糟。
将有许多看似合理的三词组合，我们可能在数据集中看不到它们。
除非我们提供某种解决方案来为这些词组合分配非零计数，否则我们无法在语言模型中使用它们。如果数据集很小或者词语非常罕见，我们可能连一个都找不到。

### 拉普拉斯平滑

一种常见的策略是执行某种形式的*拉普拉斯平滑*。
解决办法是
向所有计数添加一个小常数。
设$n$为训练集中总词数，
$m$为唯一词数。
这种解决方案有助于处理单个词，例如通过

$$\begin{aligned}
	\hat{P}(x) & = \frac{n(x) + \epsilon_1/m}{n + \epsilon_1}, \\
	\hat{P}(x' \mid x) & = \frac{n(x, x') + \epsilon_2 \hat{P}(x')}{n(x) + \epsilon_2}, \\
	\hat{P}(x'' \mid x,x') & = \frac{n(x, x',x'') + \epsilon_3 \hat{P}(x'')}{n(x, x') + \epsilon_3}.
\end{aligned}$$

这里$\epsilon_1,\epsilon_2$和$\epsilon_3$是超参数。
以$\epsilon_1$为例：
当$\epsilon_1 = 0$时，没有平滑；
当$\epsilon_1$接近正无穷时，
$\hat{P}(x)$接近均匀概率$1/m$。
上述是一种相当原始的变体，其他技术可以实现更多功能 :cite:`Wood.Gasthaus.Archambeau.ea.2011`。

不幸的是，像这样的模型很快就会变得难以处理，原因如下。
首先，
如:numref:`subsec_natural-lang-stat`所述，
许多$n$-gram很少出现，
使得拉普拉斯平滑不太适合语言建模。
其次，我们需要存储所有的计数。
第三，这完全忽略了词语的意义。例如，“cat”和“feline”应该出现在相关的上下文中。
调整这类模型以适应额外的上下文是非常困难的，
而基于深度学习的语言模型非常适合考虑到这一点。
最后，长词序列几乎肯定是新颖的，因此一个简单地
计算以前见过的词序列频率的模型在这种情况下表现必然不佳。
因此，我们在本章的其余部分将重点放在使用神经网络进行语言建模上。

## 困惑度
:label:`subsec_perplexity`

接下来，让我们讨论如何衡量语言模型的质量，我们将在后续章节中使用它来评估我们的模型。
一种方法是检查文本有多令人惊讶。
一个好的语言模型能够以高准确性预测接下来的标记。
考虑以下不同语言模型对短语“It is raining”的延续：

1. “It is raining outside”
2. “It is raining banana tree”
3. “It is raining piouw;kcj pwepoiut”

就质量而言，例1显然是最好的。这些词是有意义且逻辑一致的。
虽然它可能并不完全准确地反映哪个词会在语义上跟随（“in San Francisco”和“in winter”都是完全合理的扩展），但该模型能够捕捉到哪种类型的词会跟随。
例2明显更差，因为它产生了无意义的扩展。尽管如此，至少该模型已经学会了如何拼写单词以及一定程度的单词间关联。最后，例3表明了一个训练得不好的模型，不能很好地拟合数据。

我们可能会通过计算序列的可能性来衡量模型的质量。
不幸的是，这是一个难以理解且难以比较的数字。
毕竟，较短的序列比较长的序列更容易出现，
因此，在托尔斯泰的大作《战争与和平》上评估模型不可避免地会产生比圣埃克苏佩里的小说《小王子》更小的可能性。缺少的是相当于平均值的东西。

信息论在这里派上了用场。
当我们介绍softmax回归时定义了熵、惊奇度和交叉熵
(:numref:`subsec_info_theory_basics`)。
如果我们想压缩文本，我们可以询问
根据当前的一组标记预测下一个标记。
更好的语言模型应该允许我们更准确地预测下一个标记。
因此，它应该允许我们花费较少的比特来压缩序列。
因此，我们可以通过整个序列$n$个标记的交叉熵损失平均值来衡量它：

$$\frac{1}{n} \sum_{t=1}^n -\log P(x_t \mid x_{t-1}, \ldots, x_1),$$
:eqlabel:`eq_avg_ce_for_lm`

其中$P$由语言模型给出，$x_t$是在时间步$t$从序列中观察到的实际标记。
这使具有不同长度的文档的表现具有可比性。出于历史原因，自然语言处理领域的科学家们更喜欢使用一个称为*困惑度*的数量。简而言之，它是:eqref:`eq_avg_ce_for_lm`的指数：

$$\exp\left(-\frac{1}{n} \sum_{t=1}^n \log P(x_t \mid x_{t-1}, \ldots, x_1)\right).$$

困惑度最好理解为几何平均数的倒数，当我们决定选择下一个标记时实际拥有的真实选择数量。让我们看几个案例：

* 在最佳情况下，模型总是完美地估计目标标记的概率为1。在这种情况下，模型的困惑度为1。
* 在最坏的情况下，模型总是预测目标标记的概率为0。在这种情况下，困惑度为正无穷大。
* 在基准情况下，模型预测词汇表中所有可用标记的均匀分布。在这种情况下，困惑度等于词汇表中唯一标记的数量。实际上，如果我们不进行任何压缩存储序列，这是我们可以做的最好的编码方式。因此，这提供了任何有用的模型必须击败的一个非平凡的上限。

## 序列划分
:label:`subsec_partitioning-seqs`

我们将使用神经网络设计语言模型
并使用困惑度来评估
模型在给定当前一组标记时预测文本序列中下一个标记的好坏。
在介绍模型之前，
假设它一次处理一批具有预定义长度的序列。
现在的问题是如何[**随机读取输入序列和目标序列的小批量**]。

假设数据集采用`corpus`中$T$个标记索引的形式。
我们将
将其划分为子序列，每个子序列有$n$个标记（时间步）。
为了每次迭代
(几乎)整个数据集的所有标记
并在每个epoch中获取所有可能的长度为$n$的子序列，
我们可以引入随机性。
更具体地说，
在每个epoch的开始，
丢弃前$d$个标记，
其中$d\in [0,n)$是随机均匀采样的。
剩下的序列
然后被划分为$m=\lfloor (T-d)/n \rfloor$个子序列。
记$\mathbf x_t = [x_t, \ldots, x_{t+n-1}]$为从时间步$t$开始的长度为$n$的子序列$x_t$。
得到的$m$个划分后的子序列为
$\mathbf x_d, \mathbf x_{d+n}, \ldots, \mathbf x_{d+n(m-1)}$。
每个子序列将作为输入序列进入语言模型。

对于语言建模，
目标是基于已看到的标记预测下一个标记；因此目标（标签）是原序列，向前移动一个标记。
任何输入序列$\mathbf x_t$
的目标序列是长度为$n$的$\mathbf x_{t+1}$。

![从长度为5的划分子序列中获得五对输入序列和目标序列。](../img/lang-model-data.svg) 
:label:`fig_lang_model_data`

:numref:`fig_lang_model_data`显示了$n=5$和$d=2$时从划分的长度为5的子序列中获得五个输入序列和目标序列对的例子。

In [2]:
@d2l.add_to_class(d2l.TimeMachine)  #@save
def __init__(self, batch_size, num_steps, num_train=10000, num_val=5000):
    super(d2l.TimeMachine, self).__init__()
    self.save_hyperparameters()
    corpus, self.vocab = self.build(self._download())
    array = torch.tensor([corpus[i:i+num_steps+1]
                        for i in range(len(corpus)-num_steps)])
    self.X, self.Y = array[:,:-1], array[:,1:]

为了训练语言模型，
我们将随机抽取
输入序列和目标序列的配对
以小批量的形式。
以下的数据加载器每次都会从数据集中随机生成一个小批量。
参数 `batch_size` 指定了每个小批量中的子序列示例数量，
而 `num_steps` 则是子序列的长度（以标记为单位）。

In [3]:
@d2l.add_to_class(d2l.TimeMachine)  #@save
def get_dataloader(self, train):
    idx = slice(0, self.num_train) if train else slice(
        self.num_train, self.num_train + self.num_val)
    return self.get_tensorloader([self.X, self.Y], train, idx)

如下所示，
目标序列的小批次
可以通过
将输入序列
向后移动一个标记来获得。

In [4]:
data = d2l.TimeMachine(batch_size=2, num_steps=10)
for X, Y in data.train_dataloader():
    print('X:', X, '\nY:', Y)
    break

X: tensor([[10,  4,  2, 21, 10, 16, 15,  0, 20,  2],
        [21,  9,  6, 19,  0, 24,  2, 26,  0, 16]]) 
Y: tensor([[ 4,  2, 21, 10, 16, 15,  0, 20,  2, 10],
        [ 9,  6, 19,  0, 24,  2, 26,  0, 16,  9]])


## 摘要与讨论

语言模型估计文本序列的联合概率。对于长序列，$n$-元模型通过截断依赖关系提供了一个方便的模型。然而，由于结构复杂但频率不足，通过拉普拉斯平滑有效地处理不常见词组合的效果不佳。因此，在后续章节中我们将重点讨论神经语言建模。
为了训练语言模型，我们可以在小批量中随机抽样输入序列和目标序列对。训练后，我们将使用困惑度来衡量语言模型的质量。

通过增加数据量、模型规模和训练计算量，可以扩展语言模型。大型语言模型可以通过给定输入文本指令预测输出文本来执行所需任务。正如我们将在后面讨论的（例如，:numref:`sec_large-pretraining-transformers`），目前大型语言模型构成了多种任务中最先进系统的基础。


## 练习

1. 假设训练数据集中有100,000个单词。四元模型需要存储多少单词频率和多词相邻频率？
1. 你如何建模对话？
1. 你能想到哪些其他方法来读取长序列数据？
1. 考虑我们在每个epoch开始时丢弃一个均匀随机数量的初始标记的方法。
    1. 这真的会导致文档中的序列分布完全均匀吗？
    1. 为了让分布更加均匀，你需要做些什么？
1. 如果我们希望序列示例是一个完整的句子，这在小批量采样中会引入什么问题？我们如何解决它？

[讨论](https://discuss.d2l.ai/t/118)